In [1]:
from transformers import AutoTokenizer

# Load a pretrained BPE tokenizer 
#GPT-2	BPE (Byte-level)
#RoBERTa	BPE
#XLM-RoBERTa	BPE
#OpenAI CLIP	BPE
#T5	SentencePiece BPE

# https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaTokenizer
# Outros https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoTokenizer.from_pretrained


#Constructs a RoBERTa tokenizer, derived from the GPT-2 tokenizer, using byte-level Byte-Pair-Encoding.
#This tokenizer has been trained to treat spaces like parts of the tokens (a bit like sentencepiece) so a word will
#be encoded differently whether it is at the beginning of the sentence (without space) or not:

# Para além disto também usa ponctuação como tokens
tokenizer_bpe = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

# Example text
text =  ["The cat is crying a lot! Going chasing.  Lasted, chased, summarization"] 

# "summarization" foi dividido em 2 'Ġsummar', 'ization'
#  "Lasted" tambem 'ĠLast', 'ed'


tokens = tokenizer_bpe.tokenize(text[0])
ids = tokenizer_bpe.convert_tokens_to_ids(tokens)

print(tokens)

['The', 'Ġcat', 'Ġis', 'Ġcrying', 'Ġa', 'Ġlot', '!', 'ĠGoing', 'Ġchasing', '.', 'Ġ', 'ĠLast', 'ed', ',', 'Ġchased', ',', 'Ġsummar', 'ization']


In [ ]:
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import word_tokenize
MAX_TOKENS = 10000

tokenizer_bpe = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

def custom_tokenizer(text):
    text = text.numpy()  # Convert Tensor to NumPy array

    # Check if it's a batch of strings or a single string
    if isinstance(text, np.ndarray):  
        text = [t.decode("utf-8") if isinstance(t, bytes) else t for t in text]
    elif isinstance(text, bytes):  
        text = text.decode("utf-8")

    tokens = tokenizer_bpe.tokenize(text) # word_tokenize(text.numpy().decode("utf-8"))  # Tokenize using NLTK
    return  tf.ragged.constant(tokens)

# Wrap function to make it TensorFlow-compatible
def custom_split_fn(text):
   # return tf.py_function(func=custom_tokenizer, inp=[text], Tout=tf.string)
    return tf.py_function(func=custom_tokenizer, inp=[text], Tout=tf.RaggedTensor)

# Create TextVectorization layer
# text_vectorization_subgram = tf.keras.layers.TextVectorization(
#     max_tokens=10000,
#     output_mode="tf_idf",
#     standardize=None,  # Prevents default lowercasing and punctuation stripping
#     split=custom_split_fn,  # Custom tokenizer
# )
# text_data = tf.constant(["Hello world! NLP is fun.", "Let's build a custom tokenizer."])
# text_vectorization_subgram.adapt(text_data)  # Adapt to dataset


MAX_TOKENS = 5000 # experimentar com outros valores, 5k 10k 20k
NAME = "subgram.keras"

text_vectorization_subgram = TextVectorization(
    ngrams=1,
    max_tokens=MAX_TOKENS,
    # for int mode
    output_sequence_length=512,  # Truncate long sequences, transformers usually have 512 limit sequence
    
    output_mode="int",
    standardize=None,  # Prevents default lowercasing and punctuation stripping
    split=custom_split_fn  # Custom tokenizer
)

RESULTS[NAME] = tabular_pipeline(NAME,text_vectorization_subgram,512)
